In [7]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pathlib
import os
import copy
import datetime
import time
# sklearn?

import seaborn as sns
sns.set_style()

In [2]:
data = pd.read_csv("Data/subset-3-sous-ensemble-3.csv", encoding='latin1')

In [3]:
print(data.shape)
print(data.columns)

(1405579, 35)
Index(['LEVEL1ID', 'LEVEL2ID', 'LEVEL3ID', 'LEVEL4ID', 'LEVEL5ID', 'SURVEYR',
       'BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'DEMCODE', 'QUESTION', 'TITLE_E',
       'TITLE_F', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'DEPT_F',
       'INDICATORID', 'INDICATORENG', 'INDICATORFRA', 'SUBINDICATORID',
       'SUBINDICATORENG', 'SUBINDICATORFRA'],
      dtype='object')


In [4]:
# drop useless columns
# drop those with only 1 value: LEVEL2ID, LEVEL3ID, LEVEL4ID, LEVEL5ID
empty = []
for col in data.columns:
    uni = data[col].unique()
    if len(uni)<2:
        # print(col, uni)
        data.drop(columns=[col], inplace=True)

# DEMCODE = BYCOND, DESCRIP_E, DESCRIP_F
# QUESTION = TITLE_E, TITLE_F
# INDICATORID = INDICATORENG, INDICATORFRA
# SUBINDICATORID = SUBINDICATORENG, SUBINDICATORFRA
# LEVEL1ID = DEPT_E, DEPT_F
# SCORE5 = X*SCORE100 + Y
useless = ['BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'TITLE_E', 'TITLE_F', 'SUBINDICATORENG', 'SUBINDICATORFRA', 'DEPT_E', 'DEPT_F', 'INDICATORENG', 'INDICATORFRA', 'SCORE100']
try:
    data.drop(columns=useless, inplace=True)
except:
    pass
# drop empty lines
data.drop(data[data['ANSWER1'] == ' '].index, inplace=True)

# set ints
traduction_question = {ques: i+101 for i, ques in enumerate(data['QUESTION'].unique())}
data['QUESTION'] = data['QUESTION'].map(lambda x: traduction_question[x])

data = data.astype({'SCORE5': 'float'}, copy=False)
data = data.astype({col: 'int' for col in ['ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
    'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE',
    'ANSCOUNT']}, copy=False)

print(data.shape)
print(data.columns)

(583123, 19)
Index(['LEVEL1ID', 'SURVEYR', 'DEMCODE', 'QUESTION', 'ANSWER1', 'ANSWER2',
       'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
       'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY',
       'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE', 'SCORE5', 'ANSCOUNT',
       'INDICATORID', 'SUBINDICATORID'],
      dtype='object')


In [5]:
for col in data.columns:
    print(col, '\t', len(data[col].unique()), '\t', type(data[col][0]), '\t', data[col][0])

LEVEL1ID 	 68 	 <class 'numpy.int64'> 	 0
SURVEYR 	 3 	 <class 'numpy.int64'> 	 2020
DEMCODE 	 59 	 <class 'numpy.int64'> 	 2011
QUESTION 	 216 	 <class 'numpy.int64'> 	 101
ANSWER1 	 101 	 <class 'numpy.int32'> 	 35
ANSWER2 	 101 	 <class 'numpy.int32'> 	 45
ANSWER3 	 87 	 <class 'numpy.int32'> 	 6
ANSWER4 	 71 	 <class 'numpy.int32'> 	 11
ANSWER5 	 82 	 <class 'numpy.int32'> 	 3
ANSWER6 	 74 	 <class 'numpy.int32'> 	 0
ANSWER7 	 65 	 <class 'numpy.int32'> 	 0
MOST_POSITIVE_OR_LEAST_NEGATIVE 	 102 	 <class 'numpy.int32'> 	 81
NEUTRAL_OR_MIDDLE_CATEGORY 	 86 	 <class 'numpy.int32'> 	 6
MOST_NEGATIVE_OR_LEAST_POSITIVE 	 98 	 <class 'numpy.int32'> 	 14
AGREE 	 102 	 <class 'numpy.int32'> 	 81
SCORE5 	 366 	 <class 'numpy.float64'> 	 3.99
ANSCOUNT 	 15718 	 <class 'numpy.int32'> 	 73497
INDICATORID 	 6 	 <class 'numpy.int64'> 	 4
SUBINDICATORID 	 23 	 <class 'numpy.int64'> 	 14


In [10]:
data['ID'] =data['LEVEL1ID']*1000+(data['SURVEYR']-2018)*100+data['DEMCODE']-2011

In [52]:
list_id = data['ID'].unique()
print(len(list_id))

4992


In [86]:
data_questions = data.drop(columns = ['SURVEYR', 'DEMCODE', 'LEVEL1ID'])

In [94]:
data_agg = pd.DataFrame({'ID' : data_questions['ID'].unique()})

for i in range(101, 317):
    new_question = data_questions[data_questions['QUESTION']==i]
    new_question = new_question.drop(columns = ['QUESTION'])
    missing_id = [ID for ID in list_id if ID not in new_question['ID'].unique()]
        
    new_df = pd.DataFrame({name: ([0]*len(missing_id) if index<len(new_question.columns)-1 else missing_id) for index, name in enumerate(new_question.columns)})
    new_question = new_question.append(new_df)
            
    new_question_renamed = new_question.rename(columns=lambda x: x + "_" + str(i) if x!='ID' else x)

    data_agg = pd.merge(data_agg, new_question_renamed, on = 'ID')

print(data_agg.shape)

In [100]:
data_agg['LEVEL1ID']=data_agg['ID'] // 1000
data_agg['SURVEYR']=(data_agg['ID'] // 100) % 10 + 2018
data_agg['DEMCODE']=data_agg['ID'] % 100 + 2011

In [101]:
data_agg

,ID,ANSWER1_101,ANSWER2_101,ANSWER3_101,ANSWER4_101,ANSWER5_101,ANSWER6_101,ANSWER7_101,MOST_POSITIVE_OR_LEAST_NEGATIVE_101,NEUTRAL_OR_MIDDLE_CATEGORY_101,...,NEUTRAL_OR_MIDDLE_CATEGORY_316,MOST_NEGATIVE_OR_LEAST_POSITIVE_316,AGREE_316,SCORE5_316,ANSCOUNT_316,INDICATORID_316,SUBINDICATORID_316,LEVEL1ID,SURVEYR,DEMCODE
0,200,35,45,6,11,3,0,0,81,6,...,14,15,15,4.00,73469,6,22,0,2020,2011
1,1200,41,45,5,8,2,0,0,85,5,...,10,7,7,4.38,11552,6,22,1,2020,2011
2,2200,45,42,4,7,2,0,0,87,4,...,16,16,16,3.90,5630,6,22,2,2020,2011
3,3200,29,48,6,12,4,0,0,78,6,...,13,17,17,3.93,7308,6,22,3,2020,2011
4,4200,20,46,10,15,7,0,0,67,10,...,17,35,35,3.20,3240,6,22,4,2020,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,43153,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0,0,0,43,2019,2064
4988,46153,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0,0,0,46,2019,2064
4989,75153,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0,0,0,75,2019,2064
4990,50255,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0,0,0,50,2020,2066
